In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import gc
from collections import Counter
import copy

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [21]:
test_data = pd.read_csv("./data/test_format1.csv")
train_data = pd.read_csv("./data/train_format1.csv")
user_info = pd.read_csv("./data/user_info_format1.csv")
user_log = pd.read_csv("./data/user_log_format1.csv")

In [22]:
# 对数据内存压缩
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage after optimation is {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    return df

In [23]:
train_data = reduce_mem_usage(train_data)
test_data = reduce_mem_usage(test_data)
user_info = reduce_mem_usage(user_info)
user_log = reduce_mem_usage(user_log)

Memory usage after optimation is 1.74 MB
Decreased by 70.8%
Memory usage after optimation is 3.49 MB
Decreased by 41.7%
Memory usage after optimation is 3.24 MB
Decreased by 66.7%
Memory usage after optimation is 890.48 MB
Decreased by 69.6%


In [24]:
all_data = train_data.append(test_data)
all_data = all_data.merge(user_info, on=['user_id'], how='left')
del train_data, test_data, user_info
gc

<module 'gc' (built-in)>

In [25]:
user_log = user_log.sort_values(['user_id', 'time_stamp'])

In [26]:
list_join_func = lambda x: " ".join([str(i) for i in x])

agg_dict = {
    'item_id': list_join_func,
    'cat_id': list_join_func,
    'seller_id': list_join_func,
    'brand_id': list_join_func,
    'time_stamp': list_join_func,
    'action_type': list_join_func
}

rename_dict = {
    'item_id': 'item_path',
    'cat_id': 'cat_path',
    'seller_id': 'seller_path',
    'brand_id': 'brand_path',
    'time_stamp': 'time_stamp_path',
    'action_type': 'action_type_path'
}

def merge_list(df_ID, join_columns, df_data, agg_dict, rename_dict):
    df_data = df_data.\
        groupby(join_columns).\
        agg(agg_dict).\
        reset_index().\
        rename(columns=rename_dict)
    df_ID = df_ID.merge(df_data, on=join_columns, how='left')
    return df_ID

print(all_data.head())
all_data = merge_list(all_data, 'user_id', user_log, agg_dict, rename_dict)
print(all_data.head())

   user_id  merchant_id  label  prob  age_range  gender
0    34176         3906    0.0   NaN        6.0     0.0
1    34176          121    0.0   NaN        6.0     0.0
2    34176         4356    1.0   NaN        6.0     0.0
3    34176         2217    0.0   NaN        6.0     0.0
4   230784         4818    0.0   NaN        0.0     0.0
   user_id  merchant_id  label  prob  age_range  gender  \
0    34176         3906    0.0   NaN        6.0     0.0   
1    34176          121    0.0   NaN        6.0     0.0   
2    34176         4356    1.0   NaN        6.0     0.0   
3    34176         2217    0.0   NaN        6.0     0.0   
4   230784         4818    0.0   NaN        0.0     0.0   

                                           item_path  \
0  581818 879005 581818 581818 1011673 52343 2773...   
1  581818 879005 581818 581818 1011673 52343 2773...   
2  581818 879005 581818 581818 1011673 52343 2773...   
3  581818 879005 581818 581818 1011673 52343 2773...   
4  191923 191923 191923 19192

In [27]:
del user_log
gc.collect()

1569

In [30]:
def cnt_(x):
    try:
        return len(x.split(' '))
    except:
        return -1

def nunique_(x):
    try:
        return len(set(x.split(' ')))
    except:
        return -1

def max_(x):
    try:
        return np.max([float(i) for i in x.split(' ')])
    except:
        return -1

def min_(x):
    try:
        return np.min([float(i) for i in x.split(' ')])
    except:
        return -1

def std_(x):
    try:
        return np.std([float(i) for i in x.split(' ')])
    except:
        return -1

def most_n(x, n):
    try:
        return Counter(x.split(' ')).most_common(n)[n-1][0]
    except:
        return -1

def most_n_cnt(x, n):
    try:
        return Counter(x.split(' ')).most_common(n)[n-1][1]
    except:
        return -1

def user_cnt(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(cnt_)
    return df_data

def user_unique(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(nunique_)
    return df_data

def user_max(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(max_)
    return df_data

def user_min(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(min_)
    return df_data

def user_std(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(std_)
    return df_data

def user_most_n(df_data, single_col, name, n=1):
    func = lambda x: most_n(x, n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data

def user_most_n_cnt(df_data, single_col, name, n=1):
    func = lambda x: most_n_cnt(x, n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data